# Problems in pyspark 📝🔍

PySpark là một công cụ mạnh mẽ để xử lý các bài toán dữ liệu lớn. Sau đây là một vài bài toán kinh điển:

- wordcount
- ASEAN countries count

## WordCount 📊

Cho một chuỗi, yêu cầu đếm từng ký tự trong chuỗi đó.

Solution:

In [28]:
doc = "Apache Spark is an open-source unified analytics engine for large-scale data processing. Spark provides an interface for programming clusters with implicit data parallelism and fault tolerance. Originally developed at the University of California, Berkeley's AMPLab, the Spark codebase was later donated to the Apache Software Foundation, which has maintained it since."

result = sc.parallelize([doc]) \
    .flatMap(lambda x : x.split(" ")) \
    .map(lambda x: (x, 1)) \
    .reduceByKey(lambda x, y : x + y) \
    .collect()
result

[('Apache', 2),
 ('implicit', 1),
 ('developed', 1),
 ('of', 1),
 ('codebase', 1),
 ('and', 1),
 ('later', 1),
 ('to', 1),
 ('Software', 1),
 ('has', 1),
 ('open-source', 1),
 ('unified', 1),
 ('clusters', 1),
 ('at', 1),
 ('University', 1),
 ("Berkeley's", 1),
 ('for', 2),
 ('processing.', 1),
 ('with', 1),
 ('California,', 1),
 ('it', 1),
 ('since.', 1),
 ('analytics', 1),
 ('engine', 1),
 ('fault', 1),
 ('large-scale', 1),
 ('parallelism', 1),
 ('tolerance.', 1),
 ('maintained', 1),
 ('Spark', 3),
 ('programming', 1),
 ('data', 2),
 ('Originally', 1),
 ('donated', 1),
 ('Foundation,', 1),
 ('an', 2),
 ('provides', 1),
 ('was', 1),
 ('the', 3),
 ('which', 1),
 ('is', 1),
 ('AMPLab,', 1),
 ('interface', 1)]

Giải thích chi tiết:

- `sc.parallelize` sẽ bao bọc `doc` trong `list`, đảm bảo rằng RDD có một phần tử là toàn bộ chuỗi.
- áp dụng `flatMap` lên chuỗi đó, đi kèm `split` để tách thành một `list` các từ, sau đó “làm phẳng” `list` này thành một RDD chứa các từ.
  
  Ví dụ, khi chạy thì kết quả sẽ là: 
  ```python
  ["Apache", "Spark", "is", "an", "open-source", "unified", "analytics", "engine", "for", "large-scale", "data", "processing.", "Spark", "provides", "an", "interface", "for", "programming", "clusters", "with", "implicit", "data", "parallelism", "and", "fault", "tolerance."]
  ```


- `map` chuyển đổi mỗi từ trong RDD thành một tuple có định dạng `word, count)`

    Ví dụ:

    ```python
    [("Apache", 1), ("Spark", 1), ("is", 1), ("an", 1),
    ("open-source", 1), ("unified", 1), ("analytics", 1), 
    ("engine", 1), ("for", 1), ("large-scale", 1), ("data", 1), 
    ("processing.", 1), ("Spark", 1), ("provides", 1), 
    ("an", 1), ("interface", 1), ("for", 1), ("programming", 1),
    ("clusters", 1), ("with", 1), ("implicit", 1), ("data", 1),
    ("parallelism", 1), ("and", 1), ("fault", 1), ("tolerance.", 1)]

    ```
- Sau đó, `reduceByKey` nhóm các tuple theo key word và áp dụng hàm lambda để cộng dồn count lại cho mỗi từ

    Quá trình xử lý như sau:

    + Nhóm lại:
    
        ```python
        {
        "Spark": [1, 1],
        "is": [1, 1],
        "an": [1, 1],
        "open-source": [1]
        ....................
        }
        ```
    + Áp dụng hàm lambda x, y: x + y cho mỗi nhóm:
    
        ```python
        "Spark": 1 + 1 = 2
        "is": 1 + 1 = 2
        "an": 1 + 1 = 2
        "open-source": 1 (không thay đổi vì chỉ có một phần tử)
        ......................
        ```

Kết quả là chúng ta có tần xuất với từng từ.

Đặt vấn đề: Ở bước reduce, ta thấy một vài word có count = 1 và xuất hiện 2 lần nên ta hiểu rằng x + y chính là 1 + 1. Bởi vì 2 chính là số lượng phần tử cùng key. Vậy thì nếu số lượng này lớn hơn 2 thì sao ? Chẳng hạn như:

```python
{
    "Spark": [1, 1, 1],
    "is": [1, 1],
    "an": [1, 1]
}
```
Nếu số lượng phần tử cùng key lớn hơn 2, PySpark sẽ tự động áp dụng phép toán nhiều lần để gộp tất cả giá trị về một kết quả duy nhất.

Như vậy, việc áp dụng lambda x, y: x + y cho từng nhóm sẽ được hình dung như sau:

```python
"Spark":
    1 + 1 = 2
    2 + 1 = 3

"is": 
    1 + 1 = 2
    
"an":
    1 + 1 = 2
```

Kết quả cuối cùng: ```[("Spark", 3), ("is", 2), ("an", 2)]```

## ASEAN countries count 📊

Cho một file dữ liệu `WHO-COVID-19-20210601-213841.tsv` chứa các trường sau:
- Name 
- WHO Region 
- Cases - cumulative total 
- Cases - cumulative total per 100000 population 
- Cases - newly reported in last 7 days 
- Cases - newly reported in last 7 days per 100000 population 
- Cases - newly reported in last 24 hours 
- Deaths - cumulative total 
- Deaths - cumulative total per 100000 population 
- Deaths - newly reported in last 7 days 
- Deaths - newly reported in last 7 days per 100000 population 
- Deaths - newly reported in last 24 hours 
- Transmission Classification

Dựa vào cột Cases - cumulative total , yêu cầu: 
- Đếm xem có bao nhiêu quốc gia ở khu vực Châu Á bị nhiễm Covid
- Chỉ ra quốc gia có tổng số ca lớn nhất ở khu vực này
- Lấy ra 3 quốc gia có tổng số ca nhỏ nhất

In [9]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

spark = SparkSession.builder.appName("Spark Asean count").getOrCreate()
sc = spark.sparkContext
sqlc = SQLContext(sc)

c:\Users\nam\anaconda3\envs\py3_9\lib\site-packages\pyspark\sql\context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [10]:
df = sqlc.read.csv("WHO-COVID-19-20210601-213841.tsv",
                   header=True,
                   sep="\t")

df.show()

+--------------------+--------------------+------------------------+----------------------------------------------+-------------------------------------+-----------------------------------------------------------+---------------------------------------+-------------------------+-----------------------------------------------+--------------------------------------+------------------------------------------------------------+----------------------------------------+---------------------------+
|                Name|          WHO Region|Cases - cumulative total|Cases - cumulative total per 100000 population|Cases - newly reported in last 7 days|Cases - newly reported in last 7 days per 100000 population|Cases - newly reported in last 24 hours|Deaths - cumulative total|Deaths - cumulative total per 100000 population|Deaths - newly reported in last 7 days|Deaths - newly reported in last 7 days per 100000 population|Deaths - newly reported in last 24 hours|Transmission Classification|
+-----

In [11]:
asianCount = df.where(df['WHO Region'] == 'South-East Asia') \
    .select("Cases - cumulative total") \
    .rdd.count()
     
asianCount

11

In [12]:
maxAsian = df.where(df['WHO Region'] == 'South-East Asia') \
    .select("Cases - cumulative total") \
    .rdd \
    .max()
    
maxAsian

Row(Cases - cumulative total='800,540.000')

In [17]:
three_min = df.where(df['WHO Region'] == 'South-East Asia') \
    .select("Name",'Cases - cumulative total') \
    .rdd \
    .takeOrdered(3, key=lambda x: x[0])
    
three_min 

[Row(Name='Bangladesh', Cases - cumulative total='800,540.000'),
 Row(Name='Bhutan', Cases - cumulative total='1,620.000'),
 Row(Name="Democratic People's Republic of Korea", Cases - cumulative total='0.000')]